# Dataset Overview

let check overview of the dataset.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/playground-series-s4e8/sample_submission.csv
/kaggle/input/playground-series-s4e8/train.csv
/kaggle/input/playground-series-s4e8/test.csv


In [2]:
train_df = pd.read_csv('/kaggle/input/playground-series-s4e8/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s4e8/test.csv')

In [3]:
train_df.drop_duplicates(inplace=True)

In [4]:
# train_df.head().T

In [5]:
#Target Distribution
# train_df['class'].value_counts()

In [6]:
#Data Summary
# train_df.describe(include='all').T

# Clean Dataset
Dataset contain a lot of contrived values and so let try clean and see the result.

In [7]:
#get outlier limit
def get_outlier_range(sr):
    Q3 = sr.quantile(0.75)
    Q1 = sr.quantile(0.25)
    IQR = Q3-Q1
    upper = Q3+(1.5*IQR)
    lower = Q1-(1.5*IQR)
    return lower,upper


#let define some helper function
def clean_data(df):
    #feature columns
    if 'class' in df.columns:
        df = df.drop(columns=['id','veil-type'])
        feat_cols = df.drop(columns=['class']).columns

#         modify target data to numerical 
        df['class'],_ = df['class'].factorize()
        
    else:
        df = df.drop(columns=['veil-type'])
        feat_cols = df.drop(columns='id').columns
    
    #category columns
    cat_cols = set([x if y=='object' else np.nan for x,y in zip(feat_cols,df[feat_cols].dtypes)])
    cat_cols.remove(np.nan)
    cat_cols = list(cat_cols)
    cat_cols.sort()
    
    #numerial columns
    num_cols = df[feat_cols].drop(columns=cat_cols).columns
    
    #replace missing values with np.nan
    df = df.fillna(np.nan)
    
    
    #get dictionay of relevant values 
    values ={}
    for col in cat_cols:
#         values[col] = list(df[col].value_counts().apply(lambda x : x if x >100 else np.nan).dropna().index)+['missing']
        values[col] = list(df[col].value_counts().apply(lambda x : x if x >100 else np.nan).dropna().index)+[np.nan]
        

    #modify values with non-relevant values
    for col in cat_cols:
#         df[col] = df[col].fillna('missing')
        df.loc[~df[col].isin(values[col]),col]=np.nan
    
#     #get upper_limit values
#     upper_list = {}
#     for col in num_cols:
#         _,upper_list[col]= get_outlier_range(df[col])

        
        
    #remove rows with outlier values
    for col in num_cols:
        df[col] = pd.qcut(df[col],7,duplicates='drop')
        
#     for col in feat_cols:
#         df[col],_ = pd.factorize(df[col],use_na_sentinel=True)
    

    one_hot = pd.get_dummies(df[feat_cols],dummy_na=False,dtype=np.uint8)
    df = pd.concat([df.drop(columns=feat_cols),one_hot],axis=1)


    return df


#check data status
def check_data(df):  
    
    #feature columns
    feat_cols = df.drop(columns=['class','id']).columns
    
    #category columns
    cat_cols = set([x if y=='object' else np.nan for x,y in zip(feat_cols,df[feat_cols].dtypes)])
    cat_cols.remove(np.nan)
    cat_cols = list(cat_cols)
    cat_cols.sort()
    
    
    print('### Null Value Counts ###')
    print(df.isnull().sum())
    
    print('### Feature Summary ###')
    for col in feat_cols:
        display(pd.DataFrame(df[col].value_counts()).T)
    
    print('### Numerical Columns Summary ###')
    df[list(num_cols)].describe()


# Pre-process Data

In [8]:
train  = clean_data(train_df)

In [9]:
col_list = list(train.columns)

In [10]:
train.head()

,class,"cap-diameter_(0.028999999999999998, 2.44]","cap-diameter_(2.44, 3.55]","cap-diameter_(3.55, 4.84]","cap-diameter_(4.84, 6.41]","cap-diameter_(6.41, 7.84]","cap-diameter_(7.84, 9.95]","cap-diameter_(9.95, 80.67]",cap-shape_b,cap-shape_c,...,habitat_h,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w,season_a,season_s,season_u,season_w
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [11]:
from sklearn.model_selection import train_test_split
trainX,testX,trainY,testY = train_test_split(train.drop(columns=['class']),train['class'],test_size=0.2,shuffle=True,stratify=train['class'])

## Neural Network Model

In [12]:
import tensorflow as tf

def get_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=(trainX.shape[1],)))
    model.add(tf.keras.layers.Dense(512,activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(512,activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(512,activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(loss='BinaryCrossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005), metrics=['accuracy'])
# model.compile(loss='BinaryCrossentropy', optimizer='adam', metrics=['accuracy'])
    return model



2024-08-25 19:28:31.693961: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-25 19:28:31.694055: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-25 19:28:31.699528: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [13]:
# #Train Model
# hist = model.fit(trainX,trainY,epochs=30,batch_size=256,validation_split=0.2,callbacks=[tf.keras.callbacks.EarlyStopping(patience=3,monitor='val_loss')])

# Cross Validation

In [14]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import matthews_corrcoef

skfold = StratifiedKFold(n_splits=5,shuffle=True, random_state=123)
X=train.drop(columns=['class'])
Y=train['class']

model_list=[]
score_list=[]

for fold,(train_id,test_id) in enumerate(skfold.split(X,Y)):
    trainX, trainY = X.iloc[train_id, :], Y[train_id]
    testX, testY = X.iloc[test_id, :], Y[test_id]
    model = get_model()
    model.fit(trainX,trainY,epochs=30,batch_size=256,validation_split=0.2,callbacks=[tf.keras.callbacks.EarlyStopping(patience=2,monitor='val_loss')])
    predY = model.predict(testX)
    predY = [0 if x<=0.5 else 1 for x in predY]
    mcc_score = matthews_corrcoef(testY,predY)
    model_list.append(model)
    score_list.append(mcc_score)
    print(f"--- Fold {fold+1} MCC Score: {mcc_score:.6f}")

Epoch 1/30
7793/7793 ━━━━━━━━━━━━━━━━━━━━ 150s 19ms/step - accuracy: 0.9404 - loss: 0.1454 - val_accuracy: 0.9914 - val_loss: 0.0392
Epoch 2/30
7793/7793 ━━━━━━━━━━━━━━━━━━━━ 149s 19ms/step - accuracy: 0.9907 - loss: 0.0424 - val_accuracy: 0.9920 - val_loss: 0.0371
Epoch 3/30
7793/7793 ━━━━━━━━━━━━━━━━━━━━ 152s 20ms/step - accuracy: 0.9912 - loss: 0.0402 - val_accuracy: 0.9921 - val_loss: 0.0369
Epoch 4/30
7793/7793 ━━━━━━━━━━━━━━━━━━━━ 150s 19ms/step - accuracy: 0.9916 - loss: 0.0390 - val_accuracy: 0.9922 - val_loss: 0.0365
Epoch 5/30
7793/7793 ━━━━━━━━━━━━━━━━━━━━ 150s 19ms/step - accuracy: 0.9918 - loss: 0.0382 - val_accuracy: 0.9922 - val_loss: 0.0364
Epoch 6/30
7793/7793 ━━━━━━━━━━━━━━━━━━━━ 149s 19ms/step - accuracy: 0.9919 - loss: 0.0376 - val_accuracy: 0.9923 - val_loss: 0.0362
Epoch 7/30
7793/7793 ━━━━━━━━━━━━━━━━━━━━ 150s 19ms/step - accuracy: 0.9919 - loss: 0.0378 - val_accuracy: 0.9923 - val_loss: 0.0362
Epoch 8/30
7793/7793 ━━━━━━━━━━━━━━━━━━━━ 150s 19ms/step - accuracy: 

NameError: name 'matthews_corrcoef' is not defined

# Evaluate Model

In [ ]:
# y_pred_raw = model.predict(testX)

In [ ]:
# from sklearn.metrics import matthews_corrcoef
# y_pred = [0 if x<=0.5 else 1 for x in y_pred_raw]
# matthews_corrcoef(testY,y_pred)

In [ ]:
# from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
# import seaborn as sns
# import matplotlib.pyplot as plt
# cf = confusion_matrix(testY, y_pred)
# sns.set_context('talk')
# disp = ConfusionMatrixDisplay(confusion_matrix=cf)
# disp.plot()
# plt.show()

# Submission

In [ ]:
#pre-process data
test = clean_data(test_df)

In [ ]:
test.head()

In [ ]:
pred =[]
for model in model_list:
    pred.append([0 if x<=0.5 else 1 for x in model.predict(test.drop(columns=['id'])).squeeze()])

In [ ]:
sub = pd.DataFrame(['p' if x==1 else 'e' for x in np.median(pred,axis=0) ],index=test['id'],columns=['class'])
sub.to_csv('submission.csv')

In [ ]:
!head submission.csv